In [1]:
#Do not install gym globally, causes many reference errors and crashes the kernal
!pip install numpy --upgrade
!pip install tensorflow
%pip install -U gym>=0.21.0
%pip install -U gym[atari,accept-rom-license]
!pip install pyglet
!pip install keras-rl2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent, CEMAgent, SARSAAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [7]:
#DO NOT TOUCH THE NUMBERS
#Tensor
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), strides=(1,1) activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    # V This may be uneccesary V
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

#Tensor
#Model with the 256 dense filter removed, doesn't seem to change much, might be useful for large number of steps
def build_model_O(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512), activation='relu')
    model.add(Dense(actions), activation='linear')
    return model

def build_agent(model, actions):
    #Epislon Greedy implimentation, epislon goes from max to min over the steps to make sure it doesn't stop doing random inputs, test can be changed .1 or .2 seem to work best
    #Tensor
    
    #Just using Epsilon
    #policy = EpsGreedyQPolicy()
    
    #Using LinearAnnealment
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    #rl
    #Counts EPISODES not training frames
    memory = SequentialMemory(limit=1000, window_length=3)
    #rl
    #If doing smaller runs, smaller warmup steps works better
    dqn = DQNAgent(model=model, policy=policy, memory=memory, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=100)
    return dqn

In [8]:
#gym
env = gym.make('Seaquest-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n
#CNN model compiler, check model.summary if something goes wrong
model = build_model(height, width, channels, actions)
#Google is wrong, use .0001, not .000025, takes forever
optimizer = Adam(learning_rate=.0001)

In [9]:
#Only used for multi tier training
total_steps = 0
#The number of steps to train for in one session
next_steps = 5000
#Auto updates filepath to save multiple segments of the same run
file_path = 'Weights/'+str(total_steps + next_steps)+'DQN.h5f'
#This is prone to crashing, delete and rebuild the model to fix most issues
dqn = build_agent(model, actions)
#Optimizer Adam declared with other variables at the top of the program
dqn.compile(optimizer)
#Main training portion this takes a LONG time to complete and uses a lot of PC resources
dqn.fit(env, nb_steps=next_steps, action_repetition=1, visualize=False, verbose=1, log_interval=1000)
#Saves weights to path
dqn.save_weights(file_path)

Training for 5000 steps ...
Interval 1 (0 steps performed)


H:\Users\Greg\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


1000/1000 [==============================] - 1488s 1s/step - reward: 0.1400
1 episodes - episode_reward: 80.000 [80.000, 80.000] - loss: 11.679 - mean_q: 21.440 - mean_eps: 0.950 - lives: 3.097

Interval 2 (1000 steps performed)
1000/1000 [==============================] - 1677s 2s/step - reward: 0.1200
1 episodes - episode_reward: 100.000 [100.000, 100.000] - loss: 0.419 - mean_q: 20.671 - mean_eps: 0.865 - lives: 2.159

Interval 3 (2000 steps performed)
1000/1000 [==============================] - 1694s 2s/step - reward: 0.1600
1 episodes - episode_reward: 160.000 [160.000, 160.000] - loss: 0.414 - mean_q: 21.005 - mean_eps: 0.775 - lives: 2.601

Interval 4 (3000 steps performed)
 268/1000 [=======>......................] - ETA: 22:39 - reward: 0.0746done, took 5356.424 seconds


In [22]:
#Testing portion
#visualize=True only works locally, DOES NOT WORK on Google Colab
test_scores = dqn.test(env, nb_episodes=2, visualize=True)
env.close()
print(np.mean(test_scores.history['episode_reward']))

Testing for 2 episodes ...
Episode 1: reward: 140.000, steps: 987
Episode 2: reward: 120.000, steps: 706
130.0


In [16]:
#Loads saved weights
dqn.load_weights('Weights/5kDQN.h5f')

In [5]:
#Manually saves current environmental weights to path
dqn.save_weights('Weights/this.h5f')

NameError: name 'dqn' is not defined